# Lecture 8: Exploring Tabular Data

Using the example of metadata we have a closer look at working with CSV files (and tabular data more generally). We

Precisely, in this lecture we cover:

- load as .CSV as a pandas dataframe
- selecting rows
- manipulating values
- sorting dataframes
- plotting data


## 8.1 Introduction

In this lecture we turn to working with (semi-)structured data. 

We referred to text as 'unstructured' because it Python initially reads the document as sequence of characters. Most of our effort went to wrangling 'raw' text to more meaningful representations, by for example detecting and counting words.

In the coming lectures, we will insepct tabular or structured data. Tabular data consists of rows and columns. The rows represent individual records, which can be basically anything, a book, a measurement, a person. The columns are the atributes of these records. A common format of tabular data are spreadsheets, which you can open and edit with programs such as Microsoft Excel.

Without further ado, let's look at a concrete example: structured metadata on British Library book corpus.

## 8.2 CSV Data: Metdata on the British Library Books Corpus


The British Library Book corpus (BLB) contains work dating primarily from the 19th century. With rights cleared, this corpus is freely accessible to researchers and has proved a rich resource for previous and ongoing research projects. 

One problems with this corpus, however, is its composition. The selection criteria remain somewhat of a mystery: understanding the contours of the corpus is a non-trivial task and requires additional research at the level of corpus metadata.

In this lecture we demonstrate how to explore the BLB metadata and get a better grip on this corpus. 

The data is available by following this link: `https://bl.iro.bl.uk/downloads/e1be1324-8b1a-4712-96a7-783ac209ddef?locale=en`. Let's inspect its basic format, before we explore the topic of dataframes in more detail.

In the code below we use the `requests` library to download the data and  save it in the 'data`. We then print the first 300 characters.

In [ ]:
import requests
link = 'https://bl.iro.bl.uk/downloads/e1be1324-8b1a-4712-96a7-783ac209ddef?locale=en'
data = requests.get(link).text


In [ ]:
data[:400]

As you notice this data is actually just text, i.e. the metadata is initially just a string. We can confirm this by printing the data `type`.

In [ ]:
type(data)

But ho, wait. Didn't you tell us previousl we'd be working with structured data? Yes, but let's have a look at the data in its 'raw' format. 

What we printed earlier are the column names. You can observe how each name is separated by a comma. Also, spot the return character `\n` this the end of the line. 

While initially just a text file, you notice that BLB has an implicit structure, determined by comma's (cell boundaries) and hard returns (row boundaries). This format are commonly refered to as CSV files, i.e. 'comma separated values' and you will encounter this format regularly when working with data in the Digital Humanities. 

The first row in a CSV file are commonly called the column headers and provide semantic information, i.e. what attribute of a record is recorded in this columns. 

The BL books data contains the following columns:

```
BL record ID,Type of resource,Name,Dates associated with name,Type of name,Role,All names,Title,Variant titles,Series title,Number within series,Country of publication,Place of publication,Publisher,Date of publication,Edition,Physical description,Dewey classification,BL shelfmark,Topics,Genre,Languages,Notes,BL record ID for physical resource
```

The first record looks as follow:

```
014602826,Monograph,"Yearsley, Ann",1753-1806,person,,"More, Hannah, 1745-1833 [person] ; Yearsley, Ann, 1753-1806 [person]",Poems on several occasions [With a prefatory letter by Hannah More.],,,,England,London,,1786,Fourth edition MANUSCRIPT note,,,Digital Store 11644.d.32,,,English,,003996603
```



For exemple first column (`BL record ID`) records the identifier of a record. The identifier for the first record is `014602826`.

## 8.3 Exploring CSV files as Pandas DataFrames

While you could write a script to 'parse' these data, i.e. make implicit comma-separative structure explicit—remember text.split(`,`)?—there exist quite some tools to help you exploring and analysing tabular CSV data. 

In this course we will be working with Pandas, a popular tool that powers most of data science functionalities in Python. 

Below we import Pandas using the `pd` abbraviation. This is just for convenience, to save characters. If we want to call any tools from this library we just have to just `pd` instead of `pandas`

In [ ]:
import pandas as pd

Next we can read the csv file by just providing the link to the online document.

In [ ]:
df = pd.read_csv(
    "https://bl.iro.bl.uk/downloads/e1be1324-8b1a-4712-96a7-783ac209ddef?locale=en",
    index_col='BL record ID'
)

`read_csv()` takes a string as argument. This string can either represent a path (e.g. the location of a file on your local hard drive) or a URL (e.g. a link to an online repository). In our case we provide the URL as argument. We added one more name argument `index_col` where we specified

We save the output of this function in a variable with the name `df`. The function returns a Pandas `DataFrame` object.

In [ ]:
type(df)

The dataframe consists of rows and columns. The `.shape` attribute tells you exactly the 'dimension' of the datafarame, i.e. the number of rows and colums.

In [ ]:
df.shape

As we can observe, the BLB books corpus contains 52695 items. 

To inspect the column names you can print the `.columns` attribute attached to the DataFrame object `df`. This tells us the metadata attributes present in the CSV file.

In [ ]:
df.columns

We can use the `.head()` method to print the first rows. The code below prints the first three rows

In [ ]:
df.head(3)

## !! Explain !! 

In [ ]:
df.info()

As you can see `pd.read_csv` concerted the row text to a tabular format, identifying the rows and columns.
So far we used the Pandas functionalities (the head method and attributes attached the dataframe) to explored the basic structure of the dataframe. However, its main strength lies in the many tools for accessing, manipulating and analysing content. We first discuss how to the access and retrieve content and then turn to manipulating information and deriving basic analytics. 

### 8.3.1 Access

The most straightforward method for access is via the index. In the code above we specified that `BL record ID` should be used as the index columns. This allows us the inspect the record related to this identifier. For example if we want to inspect the book with identifer `14602826` we pass this items as a string to `.loc`. 


In [ ]:
df.loc[14602831]

The syntax, as you'll notice, resembles those of dictionary, where the items between square brackets is the key via which we retrieve the corresponding value. You can read the above line in similar fashion: retrieve the record (value) with identifier (key) `14602831`. 

DataFrames also allow you to retrieve rows by their positional index using `.iloc()`. The code below prints the record at position 7 (i.e the 8th row).

In [ ]:
df.iloc[7]

`.loc` also allows for slicing (something that wasn't allows for dictionaries). The slice notation is similar to lists, where the colon separates the start and end positions. Note that the numbers in the code blow are not the positions but BL record identifiers.

In [ ]:
df.loc[14602831:14602835]

Of course,`.iloc` also allow you to slice rows from the dataframe.

In [ ]:
df.iloc[200:205]

So far, we access the content in the dataframe by specifying the rows we wanted to retrieve. But the Pandas dataframes enable you to retrieve by column, for example the one that records the date of publication for each book in our corpus.

In [ ]:
df['Date of publication']

Note that columns belong to a different data type, namely `Series`. While a DataFrame always has two dimensions (rows and columns) a Series object only has one.

In [ ]:
type(df['Date of publication'])

In [ ]:
df['Date of publication'].shape

Returning the columns itself:

In [ ]:
df['Date of publication']

The output show the BL record identifier and the corresponding year of publication for that book. Please note the following about:
- Firstly, some records have NaN (not a number) as date. This points to missing data, i.e. the book lacks a date of publication which can happen for many reasons. In Pandas the NaN is an instance of the float class. Run the code below to see if for yourself.

In [ ]:
n = df.loc[16289059,'Date of publication']
print(n)

In [ ]:
type(n)

- Secondly, the returned column, comes with specification of its data type or `dtype`. On this case, the date of publication columns has `object` a type, which often that the columns contains information of different types. This may come a surprise, as we would expect dates or integers to appear in this column. If we look closer at the row with id `16289061` we observe that years are read as strings. In other words `Date of Publication` contains a mixture of string and float objects. Later in this tutorial we will show how to convert information in this columns to a integer indicating the year of publication, based in which we can plot time series.

In [ ]:
n = df.loc[16289061,'Date of publication']
print(n,type(n))

In Pandas, to select more than one column, you have to pass a list with column names (note the double opening and closing brackets in the statement below).

In [ ]:
df[['Date of publication','Genre']]

In [ ]:
df['Genre'].value_counts()

We can inspect the content of each column using other methods provided by the Pandas toolkit. When applied to a Series object, the `.unique()` methods show the set of the values in a columns (i.e. each unique value). This often helps with understanding and exploring the content of the CSV file. For example below we can inspect all the genres present in the BL books corpus.

In [ ]:
df['Genre'].unique()

We can then use this information to select rows based on their genre, e.g. return all books categorized as 'Travel' literature. To accomplish this we need to construct a **mask** which is an array of boolen (True, False) values that express if the row matches our condition. Let's explore this with a toy-example.

First we create a new, small, dataframe, which only records date of publication and genre.

In [ ]:
df_toy = pd.DataFrame([[1944,'Travel'],
          [1943,'Periodical'],
          [1946,'Travel'],
          [1947,'Biography']], columns= ['Date of Publication','Genre'])
df_toy

The we we create a mask using the `==` (is equal to) operator. In this case we want to retrieve row where the value in the Genre column is equal to `Travel`. This returns an array (more precisely a Pandas Series object) of boolean values: True when the recorded genre of a book matches the string `"Travel"`, False otherwise.

In [ ]:
df_toy['Genre'] == "Travel"

We can save this mask in the `mask` variabel (note difference between `=`, value assignment, and `==` equals operator )

In [ ]:
mask = df_toy['Genre'] == "Travel"
mask

Then we can pass `mask` to loc, which returns those rows in the toy dataframe that contain travel literature.

In [ ]:
df_toy.loc[mask]

Masking will return later on this course. For now it suffices to say that Pandas provides some useful functions for selecting subsets of a of dataframe. `.isin()` for example, is useful in scenarios where one wants to find multipe genres, for example `'Travel'` and `'Biography'`. This method takes a list of values as argument, and will return rows whose values appear in this list.

In [ ]:
mask = df_toy['Genre'].isin(["Travel","Biography"])
mask

In [ ]:
df_toy[mask]

Of course, we could have repeated the technique with  `==` operator and later combine the results, but `.isin()` provides a more elegant solution.

Returning now to to the main example, the BLB corpus. The statements below demonstrate how masking enables you to explore these data by Genre. 

Note how we save the subsection of the original dataframe in a new variable `travel`.

In [ ]:
mask = df['Genre'] == "Travel"
travel = df[mask]
travel

After these steps, we can work on this specific set of rows and inspect the number of Travel books in the collection and their titles.

In [ ]:
travel.shape

In [ ]:
travel['Title']

In [ ]:
travel['Title'].iloc[2]

There is one more symbol we'd like to introduce before turning again to the leading example, the tilde or `~` which basically serves a negation: in the case below, it will return all rows except those having 'Travel' as their main Genre.

In [ ]:
mask = df_toy['Genre'].isin(["Travel"])
df_toy[~mask]

### 8.3.2 Manipulating dataframes

We have already covered quite some ground in this lesson. At this point, you should have some basic understanding of how to open and explore Pandas DataFrames. In the part we go one step further demonstrate how you can change and manipulate information in these dataframes. We focus on the example of processing the dates of publication, converting the strings to integers that indicate the year of publication. This will help us later on with plotting and investigating trends over time in this corpus.

First, let us inspect the values in this columns in more detail.

In [ ]:
df['Date of publication'].unique()

The messines of these data are quite common when working with heritage collections. Even though the data is structured, it still requires some processing

In [ ]:
df['Date of publication'] > 1850

In [ ]:
df['Date of publication'].value_counts()

In [ ]:
df = df[~df['Date of publication'].isnull()]
df.shape

In [ ]:
int('2016')

In [ ]:
'2000-2019'.split('-')[0]

In [ ]:
int('2000-2019'.split('-')[0])

In [ ]:
first_year = lambda x: int(x.lstrip('-').split('-')[0])

In [ ]:
first_year('2016')

In [ ]:
first_year('2000-2019')

In [ ]:
df.loc[:,'First year of pulication'] = df['Date of publication'].apply(first_year)

In [ ]:
df['First year of pulication'].unique()

In [ ]:
df['First year of pulication'].value_counts().sort_index().plot(kind='bar',figsize=(20,5))

In [ ]:
df['First year of pulication'].value_counts().sort_index()[1800:1900]#.plot(kind='bar',figsize=(20,5))

In [ ]:
df['First year of pulication'].value_counts().sort_index().loc[1800:1900]#.plot(kind='bar',figsize=(20,5))

In [ ]:
df['First year of pulication'].value_counts().sort_index().loc[1800:1900].plot(kind='bar',figsize=(20,5))

In [ ]:
%matplotlib inline
df['Genre'].value_counts()[:10].plot(kind='bar',figsize=(20,5))

In [ ]:
df['First year of pulication'] > 1850

In [ ]:
df[df['First year of pulication'] > 1850]

In [ ]:
list(range(1850,1860))

In [ ]:
df['First year of pulication'].isin(range(1950,1956))

In [ ]:
df[df['First year of pulication'].isin(range(1850,1860))]

In [ ]:
df[(df['First year of pulication'] > 1900) & (df['Languages'] != 'English')]

In [ ]:
df[(df['First year of pulication'] > 1900) & (df['Languages'] == 'German')]['Title']

In [ ]:
import sklearn.feature_extraction.text as text

In [ ]:
df = df[~df['Title'].isnull()]

In [ ]:
df_en = df[df.Languages=='English']

In [ ]:
vectorizer = text.CountVectorizer(min_df=10,stop_words='english')
dtm = vectorizer.fit_transform(df_en['Title'])

In [ ]:
dtm.shape

In [ ]:
print(f'Shape of document-term matrix: {dtm.shape}. '
f'Number of tokens {dtm.sum()}')

In [ ]:
vocabulary = vectorizer.get_feature_names()

In [ ]:
vocabulary[100:110]

In [ ]:
dtm_df = pd.DataFrame(dtm.toarray(),index=df_en.index,columns=vocabulary)

In [ ]:
dtm_df.head(3)

In [ ]:
data = df_en.merge(dtm_df, right_index=True, left_index=True)

In [ ]:
by_year = data.groupby('First year of pulication').sum()#['chicago'].sum()#.loc[1950:].plot(kind='bar')

In [ ]:
yearly_counts = by_year.sum(axis=1)

In [ ]:
yearly_probs = by_year.divide(yearly_counts,axis=0)

In [ ]:
yearly_probs['woman']

In [ ]:
yearly_probs['woman'].loc[1800:1900].plot(kind='bar',figsize=(20,5))